In [26]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [28]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import ssl
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Embedding 

import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [29]:
#download's the stop word's
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/calebschaefer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
dataset = load_dataset("yelp_review_full")
dp_train = dataset['train'].to_pandas()
dp_test = dataset['test'].to_pandas()
dp_train

  0%|          | 0/2 [00:00<?, ?it/s]

,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...
...,...,...
649995,4,I had a sprinkler that was gushing... pipe bro...
649996,0,Phone calls always go to voicemail and message...
649997,0,Looks like all of the good reviews have gone t...
649998,4,I was able to once again rely on Yelp to provi...


In [31]:
#preprocess's the text data
#by removing stop words and leading spaces 
#this data pre-process was inspired by: https://www.kaggle.com/code/gcdatkin/gru-hotel-rating-prediction
def pre_process_data(X):
    stop_words = stopwords.words('english')
    X = re.sub(r'\d+', ' ', X)
    X = X.split()
    X = " ".join([word for word in X if word.lower().strip() not in stop_words])
    return X

In [32]:
reviews = dp_train['text'].apply(pre_process_data)
reviews

0         dr. goldberg offers everything look general pr...
1         Unfortunately, frustration Dr. Goldberg's pati...
2         going Dr. Goldberg years. think one st patient...
3         Got letter mail last week said Dr. Goldberg mo...
4         know Dr. Goldberg like moving Arizona, let tel...
                                ...                        
649995    sprinkler gushing... pipe broken way ground, t...
649996    Phone calls always go voicemail messages retur...
649997    Looks like good reviews gone head place! Jason...
649998    able rely Yelp provide needed response leaking...
649999    using company months. Ryan would come every we...
Name: text, Length: 650000, dtype: object

In [33]:
#num_words = 15000

#takes the top 10,000 most used word's so we don't have the entire vocab
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)

sequences = tokenizer.texts_to_sequences(reviews)

In [34]:
#get the length of the largest sequnces
max_length = np.max(list(map(lambda x: len(x), sequences)))
#pad all the input's to be the same length of the max length
print(max_length)
inputs = pad_sequences(sequences, maxlen=max_length, padding = 'post')
labels = np.array(dp_train['label'])
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, train_size=0.80, random_state=100)

1166


In [35]:
#Tensorlflow mlp was inspired from their website
# ttps://www.tensorflow.org/guide/core/mlp_core and #https://www.geeksforgeeks.org/multi-layer-perceptron-learning-in-tensorflow
#input_shape = tf.keras.Input(shape=(max_length,))
#We want to use an embedding so the neural network is better able to differniate words
model = Sequential([
    #set's the input to be the size of the text (which is the max-length)
    Embedding(input_dim = num_words, output_dim = 256),
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    Dense(256, activation = 'relu'),
    
    #output layer
    Dense(5, activation = 'softmax'),
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 4816s 296ms/step - accuracy: 0.5088 - loss: 1.0918
Epoch 2/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 6092s 375ms/step - accuracy: 0.6447 - loss: 0.8211
Epoch 3/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 9319s 573ms/step - accuracy: 0.7603 - loss: 0.5777
Epoch 4/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 4771s 294ms/step - accuracy: 0.8452 - loss: 0.3883
Epoch 5/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 5889s 362ms/step - accuracy: 0.8930 - loss: 0.2749
Epoch 6/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 5997s 369ms/step - accuracy: 0.9223 - loss: 0.2043
Epoch 7/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 5915s 364ms/step - accuracy: 0.9408 - loss: 0.1570
Epoch 8/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 5915s 364ms/step - accuracy: 0.9534 - loss: 0.1249
Epoch 9/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 4815s 296ms/step - accuracy: 0.9625 - loss: 0.1006
Epoch 10/10
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 4776s 294ms/step - accuracy: 0.9689 - loss: 0.0850


In [37]:
tf.saved_model.save(model, "/Users/calebschaefer/Documents/comp_5600/term_project/model3")

INFO:tensorflow:Assets written to: /Users/calebschaefer/Documents/comp_5600/term_project/model3/assets


INFO:tensorflow:Assets written to: /Users/calebschaefer/Documents/comp_5600/term_project/model3/assets


In [38]:
model.evaluate(X_test, y_test)

4063/4063 ━━━━━━━━━━━━━━━━━━━━ 93s 23ms/step - accuracy: 0.5292 - loss: 3.4884


[3.5062315464019775, 0.526761531829834]

In [20]:
test_reviews = dp_test['text'].apply(pre_process_data)
test_reviews

0        got 'new' tires within two weeks got flat. too...
1        waste time. two different people come house gi...
2        say worst! people place lunch, place freezing ...
3        restaurant twice disappointed times. go back. ...
4        Food GOOD all! husband & ate couple weeks ago ...
                               ...                        
49995    wanted write review chip others said. would re...
49996    Great ambience. Great drinks. Great food. love...
49997    Monks locations excited heard coming Sun Prair...
49998    go here. know might want try good reviews peop...
49999    Buffet recently open renovation husband thinki...
Name: text, Length: 50000, dtype: object

In [35]:
sequences = tokenizer.texts_to_sequences(test_reviews)
#max_length = np.max(list(map(lambda x: len(x), sequences)))
#print(max_length)
#pad all the input's to be the same length of the max length
test_inputs = pad_sequences(sequences, maxlen=855, padding = 'post')
test_inputs

array([[  17, 2237,  721, ...,    0,    0,    0],
       [ 820,    8,   46, ...,    0,    0,    0],
       [  65,  352,   28, ...,    0,    0,    0],
       ...,
       [1220,  641,  496, ...,    0,    0,    0],
       [  12,   63,   41, ...,    0,    0,    0],
       [ 235,  727,  236, ...,    0,    0,    0]], dtype=int32)

In [36]:
y_test = np.asarray(dp_test['label'])
print(y_test.shape)
print(test_inputs.shape)

(50000,)
(50000, 855)


In [37]:
model.evaluate(test_inputs, y_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.4919 - loss: 2.3134


[2.275451898574829, 0.5008999705314636]